### **IMPORTING LIBRARIES**

In [6]:
#Resnet as Fixed Feature Extractor

from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets,transforms,models
import torchvision.transforms as transforms
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from torch.autograd import Variable
from PIL import Image
from __future__ import print_function,division
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
from torch import optim
import time
import os
import copy
import matplotlib.pyplot as plt
cudnn.benchmark=True
plt.ion()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### **DATA NORMALIZATION WITH IMAGENET MEAN AND STD**


In [8]:
data_transforms={'train':transforms.Compose([transforms.RandomResizedCrop(224)
,transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]),
'test':transforms.Compose([transforms.Resize(256)
,transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
}


**Loading Train and Test data using dataloader**

In [9]:
data_dir = '/content/drive/MyDrive/FT_1/assignment1_data/classification_dataset/'

image_datasets={x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','test']}

dataloaders={x: torch.utils.data.DataLoader(image_datasets[x], batch_size=532,shuffle=True,num_workers=2) for x in ['train','test']}

dataset_sizes={x: len(image_datasets[x]) for x in ['train','test']}

class_names=image_datasets['train'].classes

print(dataset_sizes)

print(class_names)

{'train': 532, 'test': 120}
['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']


### **LOADING RESNET**

In [10]:
#Loading Resnet18 model
model=torchvision.models.resnet18(pretrained=True)
nodes=get_graph_node_names(model)
print(nodes)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

(['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.add', 'layer2.1.relu_1', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.0.add', 'layer3.0.relu_1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer3.1.add', 'layer3.1.relu_1', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'la

**Collecting the features from last flatten layer(a 512 dimensional vector for each Image)**

In [13]:
return_nodes={'flatten':'l5'}
model_FE=create_feature_extractor(model,return_nodes=return_nodes)
model_FE=model_FE.to(device)

In [14]:
model_FE.eval()
with torch.no_grad():
  inputs,classes=next(iter(dataloaders['train']))
  inputs=inputs.to(device)
  features=model_FE(inputs)
  for layer_name,layer_feature in features.items():
    print(layer_name,layer_feature.shape)
    train_data_features_one=layer_feature
    train_data_classes_one=classes

l5 torch.Size([532, 512])


In [15]:
#532*512 train data features as there are 532 training images ,each with a 512 dimensional vector as feature
train_data_features_one

tensor([[0.9129, 0.6870, 1.3454,  ..., 0.1506, 1.1473, 0.5071],
        [1.7556, 0.5224, 0.3404,  ..., 0.0446, 0.7063, 0.1742],
        [0.0610, 1.7704, 1.2495,  ..., 0.0710, 0.9178, 0.5716],
        ...,
        [0.1395, 0.4687, 0.9779,  ..., 0.0488, 0.4937, 0.0170],
        [2.3035, 0.1009, 0.1535,  ..., 0.3850, 0.7634, 0.0000],
        [0.7542, 0.1819, 0.4553,  ..., 0.3978, 2.1255, 0.8106]],
       device='cuda:0')

In [16]:
#detaching from gpu to cpu
train_df=train_data_features_one.detach().cpu().numpy()
train_labels=train_data_classes_one.detach().cpu().numpy()

### **Saving the files and model for KNN.ipynb**

In [17]:
#saving the train_features and labels for KNN.ipynb
np.save('train_df.npy',train_df)
np.save('train_labels.npy',train_labels)

In [18]:
torch.save(model,'/content/drive/MyDrive/assignment1_data/classification_dataset/model' )